In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('random_forest').getOrCreate()

In [2]:
df=spark.read.csv('affairs.csv',inferSchema=True,header=True)


In [3]:
print((df.count(), len(df.columns)))


(6366, 6)


In [4]:
df.printSchema()


root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)



In [5]:
df.show(5)


+-------------+----+-----------+--------+---------+-------+
|rate_marriage| age|yrs_married|children|religious|affairs|
+-------------+----+-----------+--------+---------+-------+
|            5|32.0|        6.0|     1.0|        3|      0|
|            4|22.0|        2.5|     0.0|        2|      0|
|            3|32.0|        9.0|     3.0|        3|      1|
|            3|27.0|       13.0|     3.0|        1|      1|
|            4|22.0|        2.5|     0.0|        1|      1|
+-------------+----+-----------+--------+---------+-------+
only showing top 5 rows



In [6]:
df.describe().select('summary','rate_marriage','age','yrs_married','children','religious').show()


+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|     rate_marriage|               age|      yrs_married|          children|         religious|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|              6366|              6366|             6366|              6366|              6366|
|   mean| 4.109644989004084|29.082862079798932| 9.00942507068803|1.3968740182218033|2.4261702796104303|
| stddev|0.9614295945655025| 6.847881883668817|7.280119972766412| 1.433470828560344|0.8783688402641785|
|    min|                 1|              17.5|              0.5|               0.0|                 1|
|    max|                 5|              42.0|             23.0|               5.5|                 4|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [7]:
df.groupBy('affairs').count().show()


+-------+-----+
|affairs|count|
+-------+-----+
|      1| 2053|
|      0| 4313|
+-------+-----+



In [8]:
df.groupBy('rate_marriage').count().show()


+-------------+-----+
|rate_marriage|count|
+-------------+-----+
|            1|   99|
|            3|  993|
|            5| 2684|
|            4| 2242|
|            2|  348|
+-------------+-----+



In [9]:
df.groupBy('rate_marriage','affairs').count().orderBy('rate_marriage','affairs','count',ascending=True).show()


+-------------+-------+-----+
|rate_marriage|affairs|count|
+-------------+-------+-----+
|            1|      0|   25|
|            1|      1|   74|
|            2|      0|  127|
|            2|      1|  221|
|            3|      0|  446|
|            3|      1|  547|
|            4|      0| 1518|
|            4|      1|  724|
|            5|      0| 2197|
|            5|      1|  487|
+-------------+-------+-----+



In [10]:
df.groupBy('religious','affairs').count().orderBy('religious','affairs','count',ascending=True).show()


+---------+-------+-----+
|religious|affairs|count|
+---------+-------+-----+
|        1|      0|  613|
|        1|      1|  408|
|        2|      0| 1448|
|        2|      1|  819|
|        3|      0| 1715|
|        3|      1|  707|
|        4|      0|  537|
|        4|      1|  119|
+---------+-------+-----+



In [11]:
df.groupBy('children','affairs').count().orderBy('children','affairs','count',ascending=True).show()


+--------+-------+-----+
|children|affairs|count|
+--------+-------+-----+
|     0.0|      0| 1912|
|     0.0|      1|  502|
|     1.0|      0|  747|
|     1.0|      1|  412|
|     2.0|      0|  873|
|     2.0|      1|  608|
|     3.0|      0|  460|
|     3.0|      1|  321|
|     4.0|      0|  197|
|     4.0|      1|  131|
|     5.5|      0|  124|
|     5.5|      1|   79|
+--------+-------+-----+



In [12]:
df.groupBy('affairs').mean().show()


+-------+------------------+------------------+------------------+------------------+------------------+------------+
|affairs|avg(rate_marriage)|          avg(age)|  avg(yrs_married)|     avg(children)|    avg(religious)|avg(affairs)|
+-------+------------------+------------------+------------------+------------------+------------------+------------+
|      1|3.6473453482708234|30.537018996590355|11.152459814905017|1.7289332683877252| 2.261568436434486|         1.0|
|      0| 4.329700904242986| 28.39067934152562| 7.989334569904939|1.2388128912589844|2.5045212149316023|         0.0|
+-------+------------------+------------------+------------------+------------------+------------------+------------+



In [13]:
from pyspark.ml.feature import VectorAssembler


In [14]:
df_assembler = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children','religious'], outputCol="features")
df = df_assembler.transform(df)
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
df.select(['features','affairs']).show(10,False)


+-----------------------+-------+
|features               |affairs|
+-----------------------+-------+
|[5.0,32.0,6.0,1.0,3.0] |0      |
|[4.0,22.0,2.5,0.0,2.0] |0      |
|[3.0,32.0,9.0,3.0,3.0] |1      |
|[3.0,27.0,13.0,3.0,1.0]|1      |
|[4.0,22.0,2.5,0.0,1.0] |1      |
|[4.0,37.0,16.5,4.0,3.0]|1      |
|[5.0,27.0,9.0,1.0,1.0] |1      |
|[4.0,27.0,9.0,0.0,2.0] |1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
+-----------------------+-------+
only showing top 10 rows



In [16]:
model_df=df.select(['features','affairs'])


In [17]:
train_df,test_df=model_df.randomSplit([0.75,0.25])
print(train_df.count())
train_df.groupBy('affairs').count().show()

4796
+-------+-----+
|affairs|count|
+-------+-----+
|      1| 1561|
|      0| 3235|
+-------+-----+



In [18]:
test_df.groupBy('affairs').count().show()


+-------+-----+
|affairs|count|
+-------+-----+
|      1|  492|
|      0| 1078|
+-------+-----+



In [19]:
from pyspark.ml.classification import RandomForestClassifier
rf_classifier=RandomForestClassifier(labelCol='affairs',numTrees=50).fit(train_df)

In [20]:
rf_predictions=rf_classifier.transform(test_df)
rf_predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,17.5,0.5,0.0...|      0|[38.5820420791659...|[0.77164084158331...|       0.0|
|[1.0,22.0,2.5,0.0...|      1|[19.1376649449040...|[0.38275329889808...|       1.0|
|[1.0,22.0,2.5,0.0...|      1|[19.1376649449040...|[0.38275329889808...|       1.0|
|[1.0,22.0,2.5,1.0...|      0|[22.8669287142830...|[0.45733857428566...|       1.0|
|[1.0,27.0,2.5,0.0...|      0|[20.6508493391015...|[0.41301698678203...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[22.2241395661107...|[0.44448279132221...|       1.0|
|[1.0,27.0,6.0,2.0...|      1|[21.3837548447564...|[0.42767509689512...|       1.0|
|[1.0,27.0,13.0,2....|      1|[14.3417371549567...|[0.28683474309913...|       1.0|
|[1.0,32.0,13.0,1....|      1|[14.7629028837701...|[0.29525805767540...|    

In [21]:
rf_predictions.groupBy('prediction').count().show()


+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1304|
|       1.0|  266|
+----------+-----+



In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='affairs',metricName='accuracy').evaluate(rf_predictions)
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

The accuracy of RF on test data is 74%


In [24]:
rf_precision=MulticlassClassificationEvaluator(labelCol='affairs',metricName='weightedPrecision').evaluate(rf_predictions)
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

The precision rate on test data is 72%


In [25]:
rf_auc=BinaryClassificationEvaluator(labelCol='affairs').evaluate(rf_predictions)
print( rf_auc)

0.7651741406096806


In [26]:
rf_classifier.featureImportances

SparseVector(5, {0: 0.6282, 1: 0.0233, 2: 0.2315, 3: 0.0616, 4: 0.0554})

In [27]:
df.schema["features"].metadata["ml_attr"]["attrs"]


{'numeric': [{'idx': 0, 'name': 'rate_marriage'},
  {'idx': 1, 'name': 'age'},
  {'idx': 2, 'name': 'yrs_married'},
  {'idx': 3, 'name': 'children'},
  {'idx': 4, 'name': 'religious'}]}

In [28]:
from pyspark.ml.classification import RandomForestClassificationModel
rf_classifier.save("/home/jovyan/work/RF_model")